# Homework 5

## Problem 1

SQP

In [9]:
#pip install qpsolvers[open_source_solvers]

import torch as t

#Initializing x
x = t.tensor([[1.], [1.]]) # x[0] = x1 and x[1] = x2

#Initializing mu and W just to get calculation started -- calculated with initial conditions
u = t.tensor([[6.], [2.]]) # u[0] = u1 and u[1] = u2
W = t.tensor([[2., 0.], [0., (2 + 2*u[0] + 2*u[1])]]) 

#Calculating Lagrange Gradient
gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  )

#While loop
e = 0.003
while t.norm(gL) > e:
    #Solve for s(k), u(k+1) using QP
    #NOTE: ADD HOW I DID THIS ---------------------------------------------------------------------------------------------------------------------------------------------------- 
    g = t.tensor([[(x[1]**2 - 2*x[0])], [((x[1] - 1)**2 + 5*x[0] -15)]])
    A = t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])
    A_inv = (-14*x[1] + 4)**(-1) * t.tensor([[(2*x[1] - 2), -5.], [-2*x[1], -2. ]])
    fx = t.tensor([[2*x[0]], [(2*x[1] - 6)]])
    
    s_dum = t.matmul(-t.transpose(g, 0, 1),A_inv)
    s = t.transpose(s_dum, 0, 1) #s(k)
    
    u_first = fx + t.matmul(W, s)
    u_dum = t.matmul(-t.transpose(u_first, 0, 1),A_inv)
    u = t.transpose(u_dum, 0, 1) #mu(k+1)
    
    #Line search, alpha(k), merit function and armijo line search
    alpha = 1 #for now
    
    #x(k+1) = x(k) + alpha(k)*s(k)
    x_dum = x + alpha*s
    x = x_dum
    
    #W(k+1), BFGS
    
    
    #Calculate Lagrange Gradient
    gL = t.tensor([[2*x[0]], [2*x[1] -6]]) + t.matmul( t.tensor([[-2., 5.], [2*x[1], (2*x[1] - 2) ]])   , u  ) #need to update x and mu above


KeyboardInterrupt: 

In [14]:
from cvxopt import normal, uniform
from cvxopt.modeling import variable, dot, op, sum

#m, n = 500, 100
#A = normal(m,n)
#b = uniform(m)
#c = normal(n)

#x = variable(n)
#op(dot(c,x), A*x+sum(abs(x)) <= b).solve()

#x2 = variable(n)
#y = variable(n)
#op(dot(c,x2), [A*x2+sum(y) <= b, -y <= x2, x2 <= y]).solve()



     pcost       dcost       gap    pres   dres   k/t
 0:  7.7723e-02 -2.6332e+02  1e+03  3e+00  6e+02  1e+00
 1: -5.3899e-01 -1.7199e+01  8e+01  2e-01  4e+01  5e-02
 2: -1.0027e+00 -6.5939e+00  3e+01  7e-02  1e+01  2e-02
 3: -4.8556e-01 -2.2475e+00  5e+00  2e-02  4e+00  1e-02
 4: -3.3584e-01 -1.1589e+00  2e+00  1e-02  2e+00  6e-03
 5: -1.7405e-01 -5.6719e-01  9e-01  5e-03  8e-01  3e-03
 6: -7.6604e-02 -1.8161e-01  2e-01  1e-03  2e-01  6e-04
 7: -4.9651e-02 -9.2194e-02  1e-01  6e-04  9e-02  2e-04
 8: -4.4602e-02 -8.1224e-02  1e-01  5e-04  8e-02  2e-04
 9: -2.7922e-02 -4.3544e-02  4e-02  2e-04  3e-02  8e-05
10: -2.7201e-02 -4.1215e-02  4e-02  2e-04  3e-02  7e-05
11: -2.0931e-02 -2.7255e-02  2e-02  8e-05  1e-02  3e-05
12: -2.0413e-02 -2.6239e-02  2e-02  8e-05  1e-02  3e-05
13: -1.7324e-02 -1.9570e-02  7e-03  3e-05  5e-03  9e-06
14: -1.4995e-02 -1.5163e-02  5e-04  2e-06  4e-04  6e-07
15: -1.4822e-02 -1.4823e-02  5e-06  2e-08  4e-06  7e-09
16: -1.4820e-02 -1.4820e-02  5e-08  2e-10  4e-08  

## Problem 2

moon lander